<a href="https://colab.research.google.com/github/richychn/midas/blob/main/GPT_failing_extractions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade python-dotenv

In [ ]:
pip install llama-index

In [ ]:
pip install openai

In [24]:
pip install pathlib

In [ ]:
pip install llama-parse

In [ ]:
pip install astrapy

In [ ]:
pip install --upgrade pyenv

In [87]:
import openai
#from openai import OpenAI
#openai.organization = "org-nWxH2RSTy0MZzrPL7efXAtPs"

import llama_index
from llama_index import VectorStoreIndex, SimpleDirectoryReader, download_loader

from llama_parse import LlamaParse
from llama_index.text_splitter import SentenceSplitter
from llama_index.schema import TextNode
import re
import requests
import os
import json
from datetime import datetime
from llama_index.vector_stores import AstraDBVectorStore, VectorStoreQuery

from google.colab import userdata
from astrapy.db import AstraDB
from pathlib import Path




In [88]:
# Initialization
openai.api_key = userdata.get('OPENAI_')


db = AstraDB(
  token=userdata.get('ASTRADB_TOKEN'),
  api_endpoint=userdata.get("ASTRADB_API_ENDPOINT"))

print(f"Connected to Astra DB: {db.get_collections()}")

vector_store = AstraDBVectorStore(
            token=userdata.get('ASTRADB_TOKEN'),
            api_endpoint=userdata.get("ASTRADB_API_ENDPOINT"),
            collection_name="midas_collection",
            embedding_dimension=384,
            )

parser = LlamaParse(
    api_key=userdata.get("LLAMAPARSE_API"),
    result_type="markdown",
    verbose=True
)

Connected to Astra DB: {'status': {'collections': ['midas_collection']}}


<ipython-input-88-caf699423965>:11: UserWarning: Collection 'midas_collection' is detected as legacy and has indexing turned on for all fields. This implies stricter limitations on the amount of text each entry can store. Consider reindexing anew on a fresh collection to be able to store longer texts.
  vector_store = AstraDBVectorStore(


In [135]:

path_transcript_3 = '/content/drive/MyDrive/Midas - LlamaIndex Hackathon/Email PDFs/4__2024_02_03_08_10_pm.rtf'
path_transcript_2 = '/content/drive/MyDrive/Midas - LlamaIndex Hackathon/Email PDFs/4__2024_02_03_07_45_pm.rtf'
path_transcript_1 = '/content/drive/MyDrive/Midas - LlamaIndex Hackathon/Email PDFs/4__2024_02_03_07_28_pm.rtf'

path_email = '/content/drive/MyDrive/Midas - LlamaIndex Hackathon/Email PDFs/LlamaJacket_Email.pdf'

path_list = [path_transcript_1, path_transcript_2 ,path_transcript_3 ,path_email]

In [136]:
docs_list = SimpleDirectoryReader(input_files=path_list).load_data()

In [124]:
full_llama_jacket_doc = ''
for i in range(len(docs_list)):
  full_llama_jacket_doc = full_llama_jacket_doc + (docs_list[i].text)

In [125]:
trimmed_llama_jacket_doc = ''
for i in range(2,8):
  trimmed_llama_jacket_doc = trimmed_llama_jacket_doc + (docs_list[i].text)

In [130]:
Prompt = '''
For each item that the buyer agreed to purchase, return the item name,
item description, quantity purchased, unit price in dollars, unit discount
in dollars, and final total price per item (quantity * (unit price - unit discount)).
Each sku of the purchased item should be treated as a separate item.
Output the result in json format following the example below.
Each line item should have its own list under lineItems.
Include all non quantity and price related details of each item under
the description key of the dictionary
{ "orderDetails":{ "orderDate": "", "orderNum": null, }, "buyer":{ "companyName": "", "pointOfContact": "", "contactEmail": "", "contactPhone": "", "shippingAddress": "", }, "lineItems":[{ "itemNumber": null, "description": "", "quantity": null, "unitPrice": null, "discount": null, “totalPrice”: null}]}
'''



In [131]:
def generate_response(context, given_prompt):
  response = openai.chat.completions.create(
      model = 'gpt-3.5-turbo'
    , messages = [{"role":'system', 'content': context + ' ' + given_prompt}])
  return response.choices[0].message.content


In [132]:
print(generate_response(full_llama_jacket_doc, Prompt))

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens. However, your messages resulted in 6499 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [133]:
print(generate_response(trimmed_llama_jacket_doc, Prompt))

{
  "orderDetails": {
    "orderDate": "February 3, 2024",
    "orderNum": null
  },
  "buyer": {
    "companyName": "LlamaIndex",
    "pointOfContact": "Kevin Smith",
    "contactEmail": "kevinsmith@llama.com",
    "contactPhone": "",
    "shippingAddress": "2755 Augustine Dr, 8th Floor, Santa Clara, CA 95054"
  },
  "lineItems": [
    {
      "itemNumber": null,
      "description": "LlamaIndex Custom Jacket - Design 3 (Cactus


In [ ]:
path_golf = '/content/drive/MyDrive/Midas - LlamaIndex Hackathon/Email PDFs/Old_DONOTUSE/Callaway Golf Bags.pdf'

In [139]:
docs_list_golf = SimpleDirectoryReader(input_files=[path_golf]).load_data()
full_golf_doc = ''
for i in range(len(docs_list_golf)):
  full_golf_doc = full_golf_doc + (docs_list_golf[i].text)


In [140]:
print(generate_response(full_golf_doc, Prompt))

{ "orderDetails":{ "orderDate": "Fri, Feb 3, 2024", "orderNum": null, }, "buyer":{ "companyName": "Golf-For-Fun", "pointOfContact": "Mohnish Shah", "contactEmail": "shahmohnish@gmail.com", "contactPhone": "", "shippingAddress": "", }, "lineItems":[{ "itemNumber": null, "description": "Callaway Premium Golf Bag", "quantity": 200, "unitPrice": 75, "discount": 0, “totalPrice”: 15000}]}

Explanation:
- The order date is mentioned as February 3, 2024.
- The buyer is Golf-For-Fun represented by Mohnish Shah.
- The contact email mentioned is shahmohnish@gmail.com.
- There is only one line item with the item description "Callaway Premium Golf Bag", a quantity of 200, a unit price of $75, no discount, and a total price of $15,000.
